# Example 6:  Simple Scheduling for Notebooks

### Author: AWS Professional Services Emerging Technology and Intelligent Platforms Group
### Date: July 23, 2021

In this example, we will demonstrate how users can simply schedule notebooks to run

In [ ]:
from aws_orbit_sdk import controller
from aws_orbit_sdk.magics.orbit import OrbitWorkbenchMagics 

In [ ]:
!rm -f /home/jovyan/shared/regression/notebooks/B-DataAnalyst/Example-1-SQL-Analysis-Athena/yyy*
!rm -f /home/jovyan/shared/regression/notebooks/B-DataAnalyst/Example-1-SQL-Analysis-Athena/xxx*
!rm -f /home/jovyan/shared/regression/notebooks/B-DataAnalyst/Example-91-Delete-DemoCronJobs/zzz*


In [ ]:
%%schedule_notebook -cron 0 */2 1/1 * *  -id cronjobcleaner
{
    "compute": {
        "node_type": "ec2",
        "container": 
            {
            "p_concurrent" :1
            },
        "podsetting":"orbit-runner-support-small"
    },
    "tasks":  [
        {
              "notebookName": "Example-92-Delete-DemoCronJobs.ipynb",
              "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
              "targetPath": "shared/regression/notebooks/B-DataAnalyst",
              "targetPrefix": "cronjobcleaner",
              "params": {}
        }
    ]
}



In [ ]:
%%run_notebook
{
    "compute": {
        "node_type": "ec2",
        "container": 
            {
            "p_concurrent" :1
            },
        "podsetting":"orbit-runner-support-small"
    },
    "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "yyy",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}


In [ ]:
%%schedule_notebook -cron 0/2 * 1/1 * ?  -id yyy
{
    "compute": {
        "node_type": "ec2",
        "container": 
            {
            "p_concurrent" :1
            },
        "podsetting":"orbit-runner-support-small"
    },
    "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "yyy",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}


In [ ]:
controller.schedule_notebooks(triggerName='xxx', frequency="0/2 * 1/1 * ?", taskConfiguration={    
    "compute": {
        "node_type": "ec2",
        "container": 
            {
            "p_concurrent" :1
            },
        "podsetting":"orbit-runner-support-small"
    },
    "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "xxx",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
    }
                           
)

In [ ]:
! date

In [ ]:
%%time 

import time
found = False
for k in range(1,20):
    print(f"waiting one more min...")
    time.sleep(60)
    found = False
    executions = controller.get_execution_history("shared/regression/notebooks/B-DataAnalyst", "Example-1-SQL-Analysis-Athena.ipynb")
    for index, row in executions.iterrows():
        print(row['relativePath'])
        if 'xxx' in row['relativePath']:
            if 'error@' in row['relativePath']:
                raise AssertionError('error in ' + row['relativePath'])
            found = True
            print(f"Found {row['relativePath']}")
    if found:
        break

if not found:
    raise AssertionError('cannot find executed notebook by schedule_notebooks()')
    
print("SUCCESS")

In [ ]:
controller.get_execution_history("shared/regression/notebooks/B-DataAnalyst", "Example-1-SQL-Analysis-Athena.ipynb")
for index, row in executions.iterrows():
    print(row['relativePath'])

In [ ]:
!date 

In [ ]:
%delete_schedule_task -id yyy
%delete_schedule_task -id xxx
